In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import metrics, model_selection, ensemble, preprocessing
import matplotlib.pyplot as plt
import tensorflow as tf

df = pd.read_csv('suumo_treated_DB.csv', sep = '\t', index_col=0)
df = df.drop([29706, 5414])
df = df.drop(df.index[df['age'] == 99])
df = df.drop(df.index[df['floor'] < -4 ])
df = df.drop(df.index[df['floor_plan'] > 9 ])
df = df.drop(df.index[(df['floor_plan'] < 3) & (df['surface'] > 400 )])
df = df.drop(df.index[(df['walk_min1'] > 100) ])

df_o = df[['floor_plan', 'plan_DK','plan_L', 'plan_K','plan_S','age','story', 'floor','surface','walk_min1', 'monthly_rent']]
ward = pd.get_dummies(df['ward'])
df_wo = pd.merge(df_o, ward, left_index=True, right_index=True)
df_wo = df_wo.dropna()
target = df_wo['monthly_rent']/100000
df_wo.drop(['monthly_rent'], axis=1, inplace=True) 
data = df_wo


df_scaler = preprocessing.StandardScaler().fit(df_wo)

target_n = target
df_n = df_scaler.transform(df_wo)

target_n = target_n.values.reshape(target_n.shape[0], 1)

In [2]:
# parameters
learning_rate = 0.002
n_hidden_1 = 256
n_hidden_2 = 128
n_hidden_3 = 32
n_input = df_n.shape[1]
n_class = 1

# declare placeholder
x = tf.placeholder(tf.float32, [None, n_input])
y_ = tf.placeholder(tf.float32, [None, 1])

# declare variables
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, stddev=0.1), name="w1"),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, stddev=0.1), name="w2"),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], 0, stddev=0.1), name="w3"), 
    'w4': tf.Variable(tf.random_normal([n_hidden_3, n_class], 0, stddev=0.1), name="w4")     
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], 0, stddev=0.1), name="b1"), 
    'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, stddev=0.1), name="b2"), 
    'b3': tf.Variable(tf.random_normal([n_hidden_3], 0, stddev=0.1), name="b3"), 
    'b4': tf.Variable(tf.random_normal([n_class], 0, stddev=0.1), name="b4")     
}

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_3, weights['w4']) + biases['b4']
    return out_layer

# train model
pred = multilayer_perceptron(x, weights, biases)

In [12]:
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, "suumo_tf.ckpt")

monthly_rent_pred = sess.run(pred, feed_dict={x: df_n}) * 100000
delta = monthly_rent_pred - target_n * 100000

INFO:tensorflow:Restoring parameters from suumo_tf.ckpt


In [21]:
monthly_rent_pred = pd.DataFrame(monthly_rent_pred)
monthly_rent_pred.columns = ['monthly_rent_pred']

In [22]:
delta = pd.DataFrame(delta)
delta.columns = ['delta']

In [24]:
df_result = df[['name', 'ward', 'age','story', 'floor','surface','station_name1', 'station_name2','station_name3','walk_min1', 'monthly_rent']]
df_result = pd.concat([df_result, monthly_rent_pred, delta], axis=1)
df_result.head()

,name,ward,age,story,floor,surface,station_name1,station_name2,station_name3,walk_min1,monthly_rent,monthly_rent_pred,delta
0,東京メトロ東西線 九段下駅 2階建 築6年,千代田,6.0,2.0,1.0,4.33,九段下駅,神保町駅,竹橋駅,1.0,40000.0,74585.974216,34585.974216
1,東京メトロ銀座線 末広町駅 5階建 築47年,千代田,47.0,5.0,5.0,9.72,末広町駅,秋葉原駅,湯島駅,5.0,57000.0,68278.169632,11278.169632
2,東京メトロ銀座線 末広町駅 5階建 築47年,千代田,47.0,5.0,5.0,9.72,末広町駅,秋葉原駅,湯島駅,5.0,57000.0,68278.169632,11278.169632
3,東京メトロ銀座線 末広町駅 5階建 築47年,千代田,47.0,5.0,5.0,9.72,末広町駅,秋葉原駅,湯島駅,5.0,57000.0,68278.169632,11278.169632
4,東京メトロ銀座線 末広町駅 5階建 築47年,千代田,47.0,5.0,5.0,9.72,末広町駅,秋葉原駅,湯島駅,5.0,57000.0,68278.169632,11278.169632


In [25]:
st = '牛込神楽坂駅'
df_out = df_result[df_result['station_name1'] == st].append(df_result[df_result['station_name2'] == st]).append(df_result[df_result['station_name3'] == st])
df_out = df_out.sort_values(by='delta', ascending=False)
#df_origin.ix[df_out.index]
df_out = df_out.ix[df_out.index]

/Users/kawaharakeisuke/.pyenv/versions/anaconda3-4.3.0/envs/scraping/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [26]:
df_out.head(40)

,name,ward,age,story,floor,surface,station_name1,station_name2,station_name3,walk_min1,monthly_rent,monthly_rent_pred,delta
22116,グランスイート神楽坂ピアースサウスヒルズ,新宿,2.0,10.0,2.0,44.02,牛込神楽坂駅,神楽坂駅,牛込柳町駅,6.0,190000.0,290199.375153,90199.375153
22048,東京メトロ東西線 神楽坂駅 5階建 築50年,新宿,50.0,5.0,3.0,58.60,神楽坂駅,牛込神楽坂駅,飯田橋駅,5.0,182000.0,262941.741943,82941.741943
27525,東京メトロ有楽町線 江戸川橋駅 12階建 築12年,文京,12.0,12.0,3.0,69.25,江戸川橋駅,神楽坂駅,牛込神楽坂駅,1.0,275000.0,362196.493149,77196.493149
21959,アリビオーレ神楽坂,新宿,16.0,21.0,5.0,41.49,神楽坂駅,飯田橋駅,牛込神楽坂駅,8.0,170000.0,229744.362831,59744.362831
21960,アリビオーレ神楽坂,新宿,16.0,21.0,9.0,75.59,神楽坂駅,飯田橋駅,牛込神楽坂駅,8.0,270000.0,229744.362831,59744.362831
27526,リビオ文京関口シティクロス＜Livio文京関口シティクロス＞,文京,12.0,12.0,3.0,69.25,江戸川橋駅,神楽坂駅,牛込神楽坂駅,1.0,275000.0,343224.811554,58224.811554
22068,都営大江戸線 牛込柳町駅 5階建 築12年,新宿,12.0,5.0,4.0,62.43,牛込柳町駅,曙橋駅,牛込神楽坂駅,6.0,242000.0,232254.815102,52254.815102
22506,パビヨンベールＹ,新宿,31.0,4.0,3.0,89.00,牛込柳町駅,神楽坂駅,牛込神楽坂駅,5.0,260000.0,309639.382362,49639.382362
22012,東京メトロ東西線 神楽坂駅 5階建 築28年,新宿,28.0,5.0,4.0,43.96,神楽坂駅,飯田橋駅,牛込神楽坂駅,5.0,175000.0,222931.098938,46931.098938
21797,コープ市ヶ谷,新宿,54.0,7.0,2.0,50.74,市ヶ谷駅,牛込神楽坂駅,神楽坂駅,3.0,165000.0,218556.666374,46556.666374
